In [1]:
#파이썬으로 gz 파일 압축 풀기
import tarfile
with tarfile.open(r'E:\Programming\python\ML\data\aclImdb_v1.tar.gz', 'r:gz') as tar:
    tar.extractall()

In [5]:
import pyprind
import pandas as pd
import os

basepath = r'E:\Programming\python\ML\data\aclImdb_v1\aclImdb'

##영화 리뷰를 한 줄씩 읽어서 데이터 프레임화 하기
labels = {'pos' : 1, 'neg' : 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file),
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]],
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:07:48


In [6]:
import numpy as np

#클래스 레이블 순서대로 데이터가 나열되있으므로 permutation 함수로 데이터 섞기
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv(r'E:\Programming\python\ML\data\movie_data.csv', index=False, encoding='utf-8')

In [7]:
#데이터 csv파일로 저장하기
df = pd.read_csv(r'E:\Programming\python\ML\data\movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [10]:
#데이터 프레임 행과 열의 개수 확인하기
df.shape

(50000, 2)

In [11]:
####Bag of Word####
#1. 전체 문서에 대해 고유한 토큰(예컨데, 단어로 이루어진 어휘 사전) 생성
#2. 특정 문서에 각 단어가 얼마나 자주 등장하는지 헤아려 문서의 특성 벡터 생성

In [13]:
###BoW 만들기
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer #CountVectorizer은 문서 또는 문장으로 이루어진 텍스트 데이터 배열을 입력 받아 BoW 모델 생성

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [15]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [16]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [17]:
###tf(단어 빈도)-idf(역문서 빈도)###
#특성 벡터에서 자주 등장하는 단어의 가중치를 낮추는 기법

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [19]:
##불필요한 문자 제거를 통한 텍스트 정제

In [23]:
#랜덤하게 섞은 데이터셋의 첫 번째 문서에서 마지막 50개의 문자를 출력
df.loc[0, 'review'][-50:] #HTML 마크업 문자와 글자가 아닌 문자 또한 포함되어있음

'is seven.<br /><br />Title (Brazil): Not Available'

In [25]:
##이모티콘은 정보가 될수 있으므로 제외하고 그 외의 모든 구두점 문자 제거
#정규표현식 사용
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) #html 마크업을 삭제하는 정규표현식
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

In [26]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [27]:
df['review'] = df['review'].apply(preprocessor)